One sample plotting tutorial
============================

Welcome to the RNAvigate tutorials. Here I will demonstrate and explain some
of the functionality of RNAvigate. In this Jupyter Notebook example, we'll
explore:

* [How to set up the environment](#how-to-set-up-the-environment)
* [How to load experimental data](#how-to-load-experimental-data)
* [How to use basic plotting functions](#how-to-use-basic-plotting-functions)

Every multi-sample plotting function takes the same form:

```python
rnav.plot_qc(samples, other_keywords="some_value")
rnav.plot_skyline(samples, other_keywords="some_value")
rnav.plot_ap(samples, other_keywords="some_value")
rnav.plot_circle(samples, other_keywords="some_value")
rnav.plot_ss(samples, other_keywords="some_value")
rnav.plot_mol(samples, other_keywords="some_value")
rnav.plot_disthist(samples, other_keywords="some_value")
rnav.plot_heatmap(samples, other_keywords="some_value")
rnav.plot_linreg(samples, other_keywords="some_value")
```

We'll dive into how to use these and what some of the other_keywords are.

Notebook set-up
---------------
First, lets set up our Jupyter Notebook, and import RNAvigate.

Let's also create a button to hide/show the raw code, this can be useful to
include if you are going to share an analysis with an audience that is not
interested in the code.

In [ ]:
import rnavigate as rnav
from data.rnasep.samples import example1, example2, example3, example4


Initializing RNAvigate samples
------------------------------

In this example, all of my files for a particular sample have the same prefix,
and are included in the same directory. This allows me to make a simple
function to create each sample. This function takes a sample prefix, prepends
the path, and appends the file suffix for each data type.

These particular samples are using different probing reagents to determine which
is working best for correlation analyses. The differences between them will be
subtle, but important.

In [ ]:
samples = [example1, example2, example3, example4]


ShapeMapper QC
--------------

The first step in any thorough analysis is quality control, so lets check the
quality of our samples against each other. In contrast to the single sample QC
plot, this one splits up untreated and modified samples.

In this plot we can see that each experiment's modified sample is shifted to the
right compared to the untreated samples, but to different degrees.

All of our samples have a good read-length distribution, and all of them have
a good shift in the distribution of per-nucleotide mutation rates.

In [ ]:
plot = rnav.plot_qc(samples, 'shapemap', 'log')


Skyline Plots
-------------

We can see that the signal magnitude is slightly different in each of our
samples, but to see how they differ qualitatively, we should plot them in detail
on the same axis. Skyline plots are the best way to show this.

In [ ]:
plot = rnav.plot_skyline(
    samples=samples,
    profile='shapemap')


By default, skyline plots the "Reactivity_profile" data, but clearly these have
different magnitudes. We want to view the normalized profiles, which we can
specify with the keyword `columns="Norm_profile"`. For the most part, these
samples are very similar, but they do have some interesting differences. Some
regions in examples 2-4 have reactivity in regions where example 1 does not.

In [ ]:
plot = rnav.plot_skyline(
    samples=samples,
    profile='shapemap',
    columns="Reactivity_profile")


Linear regressions
------------------

Another good way to compare reagents is by simple linear regression.

If a structure is provided, we can color the points on the scatter plot by
their base-pairing status. We can also get KDE curves (density curves similar to
histograms) for paired and unpaired nucleotides, to see how well our reagents
report on base-pairing. These reagents are doing pretty well. As we can see by
the pairwise R-squared values, the profiles are indeed quite different.
Typical replicate experiment R-squared values are typically greater than 0.9.

In [ ]:
plot = rnav.plot_linreg(
    samples=samples,
    profile='shapemap',
    scale='log',
    column='Reactivity_profile')


Arc Plots
---------

Where these reagents really differ is in the degree in which their correlations
detect base-pairing and tertiary structure. A good way to show this is plotting
PAIR-MaP and RING-MaP data together. First we'll filter the RINGs to be positive,
highly correlated, and distant in secondary structure.

Examples 1 and 4 are a bit messy. Example 2 picks up on an important loop-loop
interaction, but misses many of the long-range base-pairs. Example 3 reveals
the long-range pseudoknot well, but doesn't strongly report on tertiary
structure.


In [ ]:
plot = rnav.plot_arcs(
    samples,
    sequence='ss_ct',
    profile='shapemap',
    structure='ss_ct',
    interactions={
        'interactions': 'ringmap',
        "positive_only": True,
        "Statistic_ge": 23,
        "min_cd": 15,
    },
    interactions2="pairmap",
    profile_scale_factor=5)


Circle plots
------------

The same data can be displayed on circle plots, but these plots are best used
to compare a single data type at a time, as they can become cluttered quickly.

In [ ]:
plot = rnav.plot_circle(
    samples,
    sequence='ss_ct',
    interactions={
        'interactions': "ringmap",
        "positive_only": True,
        "Statistic_ge": 23,
        "min_cd": 15,
    },
    profile="shapemap",
    colors={'sequence': 'contrast',
            'nucleotides': 'profile'})


Secondary structure drawings
----------------------------

Also included in our samples is a custom secondary structure drawing. This
particular drawing is laid out to reflect the tertiary elements of this RNA,
namely, coaxially stacked helices with a complex core motif.

We can view all of the above data on these drawings as well. Here we will only
display the PAIRs that reflect base-pairing in this structure by applying the
filter `paired_only=True`.

In [ ]:
plot = rnav.plot_ss(
    samples,
    structure='ss_pdb',
    profile='shapemap',
    interactions={
        'interactions': 'ringmap',
        'positive_only': True,
        'Statistic_ge': 23,
        'min_cd': 15,
    },
    colors={'sequence': 'profile'},
    interactions2={
        'interactions': 'pairmap',
        'paired_only':True
    })


3D structures
-------------

Finally, we can view this data on the crystal structure of this RNA.

Ideally, these correlations would report on nucleotides that are close in 3D
distance. Clearly, that is not the case for these reagents. Sad.

In [ ]:
plot = rnav.plot_mol(
    samples,
    structure='pdb',
    interactions={
        'interactions': 'ringmap',
        'positive_only': True,
        'Statistic_ge': 23,
        'min_cd': 15,
    })


Distance histograms
-------------------

To finally prove to myself that these are not performing well, lets compare the
distances between nucleotides that are correlated to the pairwise distances of
all nucleotides in this crystal structure.

In [ ]:
plot = rnav.plot_disthist(
    samples,
    structure='pdb',
    interactions={
        'interactions': 'ringmap',
        'positive_only': True,
        'Zij_ge': 2,
        'min_cd': 15
    },
    rows=1)


To see how a good experiment performs on this metric, lets compare a SHAPE-JuMP
experiment.

In [ ]:
rnav.plot_disthist(
    samples[0:1],
    structure='pdb',
    interactions={
        'interactions': 'shapejump',
        'Percentile_ge': 0.99})
